# Export to file

A need for exporting dfsu files to other formats, such as raster formats (NetCDF, GeoTIFF) or vector formats (ESRI Shape, GeoJSON) are not uncommon.

There are usually very specific requirements about the exported data, naming, metadata, resolution and so on, which is why we don't provide out of the box tools to convert files back and forth from a lot of formats, but instead provide the necessary building blocks and some examples to use as inspiration.





## Examples

* [Dfsu converted to NetCdf (unstructured grid)](https://nbviewer.org/github/DHI/mikeio/blob/main/notebooks/Dfsu%20-%20Export%20to%20netcdf.ipynb)
* [Dfsu interpolated to structured grid](https://nbviewer.org/github/DHI/mikeio/blob/main/notebooks/Dfsu%20-%20Export%20to%20netcdf.ipynb)
* [Dfs2 converted to NetCDF](https://nbviewer.org/github/DHI/mikeio/blob/main/notebooks/Dfs2%20-%20Export%20to%20netcdf.ipynb)



## Export to shapefile

Exporting to GIS formats such as ESRI Shapefile or GeoTiff requires some libraries which are tricky to install on Windows.

One way is to use Conda-forge

1. Download the [Miniconda installer](https://docs.conda.io/en/latest/miniconda.html)
2. Install GeoPandas: `conda install --channel conda-forge geopandas`

An alternative is to use to run python under [Windows Subsystem for Linux](https://docs.microsoft.com/en-us/windows/wsl/install)

![](images/wsl.png)


In [ ]:
import mikeio
import geopandas as gpd

dfs = mikeio.open("data/oresundHD_run1.dfsu")
dfs

In [ ]:
ds = dfs.read(items="Surface elevation")
ds

In [ ]:
maxwl = ds["Surface elevation"].max(axis=0).values
maxwl.shape

In [ ]:
shp = dfs.geometry.to_shapely()
type(shp)

In [ ]:
poly_list = [p for p in shp.geoms]

In [ ]:
gdf = gpd.GeoDataFrame({'waterlevel': maxwl},geometry=poly_list, crs=dfs.projection_string)

Ouch... The short and smart projection string "UTM-33" is apparently not understood by GeoPandas. Better look it up on the web
https://spatialreference.org/ref/epsg/32433/

In [ ]:
gdf = gpd.GeoDataFrame({'waterlevel': maxwl},geometry=poly_list, crs="EPSG:32433")

In [ ]:
gdf.head()

Export as ESRI Shapefile

In [ ]:
gdf.to_file("waterlevel_utm.shp")

Which can be used together with other data sources and customized in QGIS
![](images/qgis_shp.png)

Or GeoJSON, which some might prefer...

In [ ]:
gdf.to_file("waterlevel_utm.json")

![](images/notepad_geojson.png)